#Import Data from the Climate Data Store:

configuration is needed in the file $HOME/.cdsapirc (in your Unix/Linux environment). 
more Information in https://cds.climate.copernicus.eu/api-how-to 

In [1]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': '2m_temperature',
        'year': [
            '2015', 
       #     '2016', '2017',
       #     '2018', '2019', '2020',
       #     '2021', '2022',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': '17:00',
    },
    'download_2015.nc')


2022-08-10 14:51:15,996 INFO Welcome to the CDS
2022-08-10 14:51:15,999 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-08-10 14:51:16,109 INFO Request is queued
2022-08-10 15:17:39,187 INFO Request is completed
2022-08-10 15:17:39,223 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data8/adaptor.mars.internal-1660137390.759536-4228-3-a7da17b3-bc82-4a4c-a48a-62eb17c324ad.nc to download_2015.nc (722.8M)
2022-08-10 15:18:53,399 INFO Download rate 9.7M/s                                                                               


Result(content_length=757926372,content_type=application/x-netcdf,location=https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data8/adaptor.mars.internal-1660137390.759536-4228-3-a7da17b3-bc82-4a4c-a48a-62eb17c324ad.nc)

In [18]:
file = '/Users/nilshempelmann/birdhouse/testdata/download_2015.nc'

### EXPERIENCE NAME
expname = "ocean_reanalysis_GREP"
#expname = "sst_retroprevision_sys7"

### PERCENTILE THRESHOLD
percent_thresh = 95
#percent_thresh = 90

### MINIMAL DURATION OF A HW
duration_min = 5
#duration_min = 3

### REGION OF EXPERIENCE
reg_name = "global"
#reg_name = 'lat30_70-lon-40_0'
#reg_name = 'lat-20_30-lon-30_10'
if reg_name == 'global':
    lats_bnds = [-90,90]
    lons_bnds = [-180,180]    
elif reg_name == 'lat30_70-lon-40_0':
    lats_bnds = [30,70]
    lons_bnds = [-40, 0]
elif reg_name == 'lat-20_30-lon-30_10':
    lats_bnds = [-20, 20]
    lons_bnds = [-30, 10]
nlat = lats_bnds[1]-lats_bnds[0]
nlon = lons_bnds[1]-lons_bnds[0]

### YEARS
if expname == 'ocean_reanalysis_GREP':
    end_year=2018
    start_year=1993
elif expname == 'sst_retroprevision_sys7':
    end_year=2016
    start_year=1993
nyear=end_year-start_year+1

### SEASON
season = "NDJFMAM"
if season == 'NDJFMAM':
    nday = 211
    season_start_day = [11,1] #1stNov
    season_end_day = [5,31] #31stMay
elif season == 'DJF':
    nday = 90
ndayseas = nday//duration_min +1

if expname == "ocean_reanalysis_GREP":    
    ### NUMBER OF MEMBS
    nmemb = 1
    chosen_memb = [0] #should never change
    memb_str = 'memb' + str(chosen_memb[0])
    
    ### CROSS VALIDATION
    cv = False
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'
        
elif expname == "sst_retroprevision_sys7":    
    ### NUMBER OF MEMBS
    nmemb = 1 #For now keep just one memb by one
    chosen_memb = [21] #to be filled in numerical order
    if len(chosen_memb) != nmemb:
        raise(ValueError)
    if nmemb>1:
        memb_str = 'memb' + str(chosen_memb[0]) + '-' + str(chosen_memb[-1])
    else:
        memb_str = 'memb' + str(chosen_memb[0])
    
    ### CROSS VALIDATION
    cv = True
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'

### PARAMETERS
parameters_str = reg_name + "_" + season + "_" + cv_str + '_percent%i'%(percent_thresh) + '_daymin%i'%(duration_min) +  "ref%i-%i"%(start_year, end_year) + '_' + memb_str

### NAME OF THE VARIABLES IN THE NC FILES
#varname = 'HWMI_' + parameters_str
varname = 'subHW_' + parameters_str

In [2]:
import matplotlib
#matplotlib.use('Agg')

In [8]:
import os
import datetime
import time
import copy
import shutil
import sys
sys.path.append('/Users/nilshempelmann/birdhouse/Heatwaves/')
# from function_read import *
# from HWs_detection import *
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap, shiftgrid


In [6]:
import calendar
import locale
locale.setlocale( locale.LC_ALL , 'en_US' )
from netCDF4 import num2date, date2num 
import netCDF4
from math import sin, cos, sqrt, atan2, radians
# from sklearn.neighbors import DistanceMetric
from math import radians

# from joblib import Parallel, delayed
# import joblib

In [20]:
from function_read import extract_array
varname = 't2m'
varf=netCDF4.Dataset(file)
varf.variables[varname]
vararray, lats_reg, lons_reg = extract_array(varf, varname, ndayseas, np.array(lons_bnds), np.array(lats_bnds),  start_time = 0, level=0)
obsreg = vararray[:, np.newaxis, :, :]

In [22]:
obsregyearslst = []
maskobs = obsreg.mask
obsregyearslst.append(obsreg)

obsregyears = np.ma.array(obsregyearslst)

In [25]:
from HW_detection import calc_HW_MY
start_time = time.time()
args = (expname, reg_name, memb_str, parameters_str, start_year, lats_reg, lons_reg)
HWampliobsmembyear, HWstartobsmembyear, HWendobsmembyear, fieldobslstmembyear = calc_HW_MY(obsregyears, maskobs, lats_reg, lons_reg, args, allowdist=1)
print('Total time for detection : ', time.time() - start_time)

ValueError: not enough values to unpack (expected 8, got 7)